In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.insert(0, "/notebooks/")
from fastai.imports import *

In [2]:
from fastai.core import V, VV_, to_np
import torch.nn.functional as F
import torch.nn as nn
from torch import zeros
import torch

n_hidden = 512
def text_model(**kwargs):
	model = CharSeqStatefulLSTM(vocab_size=55, n_fac=42, bs=64, nl=2).cuda()
	return model

class CharSeqStatefulLSTM(nn.Module):
    def __init__(self, vocab_size, n_fac, bs, nl):
        super().__init__()
        self.vocab_size,self.nl = vocab_size,nl
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.LSTM(n_fac, n_hidden, nl, dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h[0].size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [3]:
model = text_model()

In [4]:
file_path = "/notebooks/data/nietzsche/first_nietzsche"
state_dict = torch.load(file_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state_dict)

In [5]:
PATH='/notebooks/data/data/nietzsche/'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

In [6]:
from torchtext import vocab, data
from torchtext import vocab, data
from fastai.nlp import *
from fastai.lm_rnn import *


class Get():
    def __init__(self, model, inp, num):
        self.TEXT = data.Field(lower=True, tokenize=list)
        bs=64; bptt=8; n_fac=42; n_hidden=256
        FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
        self.md = LanguageModelData.from_text_files(PATH, self.TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)
        self.inp = inp
        self.num = num
        self.model = model

    def get_next(self, inp):
        idxs = self.TEXT.numericalize(inp)
        p = self.model(VV(idxs.transpose(0,1)))
        r = torch.multinomial(p[-1].exp(), 1)
        return self.TEXT.vocab.itos[to_np(r)[0]]
    
    def get_next_n(self):
        res = self.inp
        inp = self.inp
        for i in range(self.num):
            c = self.get_next(inp)
            res += c
            inp = inp[1:]+c
        return res

In [7]:
predict = Get(model, "peopl", 10)
out = predict.get_next_n()

In [8]:
out

'peoples), that '